### ingest_daily_support_tickets

In [48]:
import os
import pandas as pd
import boto3
from io import StringIO
from sqlalchemy import create_engine
from datetime import datetime, timedelta

from dotenv import load_dotenv

In [49]:
# CONFIG 
load_dotenv("sample.env")

db_config = {
    "host": "localhost",
    "port": "3306",
    "user": "root",  # change
    "password": "Heysql2628$", # change
    "database": "careplus_support_db"
}

S3_BUCKET = "care-plus-data-storage" 
S3_PREFIX = "support-tickets/raw/"  
DATE_TRACKER_FILE = "date_tracker.txt"

import os

AWS_CONFIG = {
    "aws_access_key_id": os.getenv("AWS_ACCESS_KEY_ID"),
    "aws_secret_access_key": os.getenv("AWS_SECRET_ACCESS_KEY"),
    "region_name": os.getenv("AWS_DEFAULT_REGION")
}

In [50]:
# UTILITY FUNCTIONS 
def get_engine(config):
    return create_engine(f"mysql+pymysql://{config['user']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}")

def upload_to_s3(df, bucket, key):
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)

    s3 = boto3.client('s3', **AWS_CONFIG)
    s3.put_object(Bucket=bucket, Key=key, Body=csv_buffer.getvalue())
    print(f"✅ Uploaded to s3://{bucket}/{key}")

def read_last_date(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as f:
            return f.read().strip()
    return "2025-06-30"  # Starting point before 1st July

def update_last_date(file_path, new_date):
    with open(file_path, 'w') as f:
        f.write(new_date)

def get_next_date(last_date_str):
    last_date = datetime.strptime(last_date_str, "%Y-%m-%d")
    next_date = last_date + timedelta(days=1)
    return next_date.strftime("%Y-%m-%d")

# ---------- MAIN INGESTION LOGIC ----------
def run_ingestion():
    engine = get_engine(db_config)
    last_date = read_last_date(DATE_TRACKER_FILE)
    next_date = get_next_date(last_date)

    # Query only that day’s data
    query = f"""
        SELECT * FROM support_tickets
        WHERE DATE(created_at) = '{next_date}';
    """
    df = pd.read_sql(query, engine)
    print(df.shape)
    print(df.head())

    if df.empty:
        print(f"⚠️ No data found for {next_date}. Skipping upload.")
        return

    # Upload to S3
    s3_key = f"{S3_PREFIX}support_tickets_{next_date}.csv"
    upload_to_s3(df, S3_BUCKET, s3_key)

    # Update date tracker
    update_last_date(DATE_TRACKER_FILE, next_date)
    print(f"📅 Updated tracker to {next_date}")

# Run
if __name__ == "__main__":
    run_ingestion()

(26, 10)
    ticket_id        created_at       resolved_at  agent priority  \
0  TCK0703000  2025-07-03 00:25  2025-07-03 01:58  Arjun       Lw   
1  TCK0703001  2025-07-03 01:00  2025-07-03 16:32  Sneha   Medium   
2  TCK0703002  2025-07-03 01:19  2025-07-03 12:26  Sneha       Lw   
3  TCK0703003  2025-07-03 03:00              None  Arjun    Medum   
4  TCK0703004  2025-07-03 03:14  2025-07-03 16:08  Sneha   Medium   

  num_interactions         IssUeCat   channel    status agent_feedback  
0                2  Feature Request      Chat  Resolved                 
1          -999999  Payment Failure  Web Form  Resolved                 
2                1  Feature Request     Email  Resolved                 
3                2  Feature Request  Web Form      Open                 
4                6      Login Issue      Chat  Resolved                 


/Users/neerajmn/Downloads/project-care-plus/.venv/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


✅ Uploaded to s3://care-plus-data-storage/support-tickets/raw/support_tickets_2025-07-03.csv
📅 Updated tracker to 2025-07-03
